In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
#path = "C:\\Users\\devid\\Desktop\\datasets\\Automobile_data.csv"
#path = "Automobile_data.csv"
#df = pd.read_csv(path)
# C:\Users\devid\Desktop\datasets\Automobile_data.csv" ^
# SyntaxError: (unicode error) 'unicodeescape' codec can't decode 
#df = pd.read_csv("./Automobile_data.csv")
df = pd.read_csv("Automobile_data.csv")
#df = pd.read_csv(Automobile_data.csv) #Error: 
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [3]:
# Handling missing values (replace '?' with NaN and drop missing values)
df.replace("?", np.nan, inplace=True) # replaces all occurrences of "?" in the df with NaN
df.dropna(inplace=True) # removes any rows in the DataFrame that contain NaN values

In [4]:
# Converting necessary columns to numeric
for col in [ "bore", "stroke", "horsepower", "peak-rpm", "price"]:
 df[col] = pd.to_numeric(df[col])

In [5]:
df

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
10,2,192,bmw,gas,std,two,sedan,rwd,front,101.2,...,108,mpfi,3.50,2.80,8.8,101,5800,23,29,16430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470


### Polynomial Features

In [6]:
from sklearn.preprocessing import PolynomialFeatures
# Select numerical columns
numerical_features = df[['curb-weight', 'engine-size', 'horsepower']]

In [7]:
# Create polynomial features (degree 2)
poly = PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(numerical_features)

In [8]:
poly.fit_transform(numerical_features)
poly.get_feature_names_out(numerical_features.columns)

array(['1', 'curb-weight', 'engine-size', 'horsepower', 'curb-weight^2',
       'curb-weight engine-size', 'curb-weight horsepower',
       'engine-size^2', 'engine-size horsepower', 'horsepower^2'],
      dtype=object)

In [9]:
# Create a DataFrame of new features
poly_df = pd.DataFrame(poly_features, 
columns=poly.get_feature_names_out(numerical_features.columns))

In [10]:
# Display the original and new polynomial features
print("Original Features and Polynomial Features:")
print(poly_df.head())

Original Features and Polynomial Features:
     1  curb-weight  engine-size  horsepower  curb-weight^2  \
0  1.0       2337.0        109.0       102.0      5461569.0   
1  1.0       2824.0        136.0       115.0      7974976.0   
2  1.0       2844.0        136.0       110.0      8088336.0   
3  1.0       3086.0        131.0       140.0      9523396.0   
4  1.0       2395.0        108.0       101.0      5736025.0   

   curb-weight engine-size  curb-weight horsepower  engine-size^2  \
0                 254733.0                238374.0        11881.0   
1                 384064.0                324760.0        18496.0   
2                 386784.0                312840.0        18496.0   
3                 404266.0                432040.0        17161.0   
4                 258660.0                241895.0        11664.0   

   engine-size horsepower  horsepower^2  
0                 11118.0       10404.0  
1                 15640.0       13225.0  
2                 14960.0       12100

In [11]:
poly_df.head()

,1,curb-weight,engine-size,horsepower,curb-weight^2,curb-weight engine-size,curb-weight horsepower,engine-size^2,engine-size horsepower,horsepower^2
0,1.0,2337.0,109.0,102.0,5461569.0,254733.0,238374.0,11881.0,11118.0,10404.0
1,1.0,2824.0,136.0,115.0,7974976.0,384064.0,324760.0,18496.0,15640.0,13225.0
2,1.0,2844.0,136.0,110.0,8088336.0,386784.0,312840.0,18496.0,14960.0,12100.0
3,1.0,3086.0,131.0,140.0,9523396.0,404266.0,432040.0,17161.0,18340.0,19600.0
4,1.0,2395.0,108.0,101.0,5736025.0,258660.0,241895.0,11664.0,10908.0,10201.0


### Interaction Features (Numerical Data)

In [13]:
# Create interaction feature between engine-size and curb-weight
df['engine_curb_weight_interaction'] = df['engine-size'] * df['curb-weight']

In [14]:
# Display the new feature
print("Interaction Feature (engine-size * curb-weight):")
print(df[['engine-size', 'curb-weight', 
'engine_curb_weight_interaction']].head())

Interaction Feature (engine-size * curb-weight):
    engine-size  curb-weight  engine_curb_weight_interaction
3           109         2337                          254733
4           136         2824                          384064
6           136         2844                          386784
8           131         3086                          404266
10          108         2395                          258660


In [15]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,engine_curb_weight_interaction
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,mpfi,3.19,3.4,10.0,102,5500,24,30,13950,254733
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,mpfi,3.19,3.4,8.0,115,5500,18,22,17450,384064
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,mpfi,3.19,3.4,8.5,110,5500,19,25,17710,386784
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,mpfi,3.13,3.4,8.3,140,5500,17,20,23875,404266
10,2,192,bmw,gas,std,two,sedan,rwd,front,101.2,...,mpfi,3.50,2.8,8.8,101,5800,23,29,16430,258660


### Binning (Numerical to Categorical)

In [16]:
# Create bins for horsepower
bins = [0, 100, 150, 200]
labels = ['Low', 'Medium', 'High']

In [17]:
df['horsepower_binned'] = pd.cut(df['horsepower'], bins=bins, labels=labels)

In [18]:
# Display the binned feature
print("Binned Feature (horsepower):")
print(df[['horsepower', 'horsepower_binned']].head())

Binned Feature (horsepower):
    horsepower horsepower_binned
3          102            Medium
4          115            Medium
6          110            Medium
8          140            Medium
10         101            Medium


### Encoding Categorical Features (Textual Data)

In [19]:
# One-hot encode the 'fuel-type' column
df_onehot = pd.get_dummies(df, columns=['body-style'], 
drop_first=True)
df_onehot.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,drive-wheels,engine-location,wheel-base,length,...,peak-rpm,city-mpg,highway-mpg,price,engine_curb_weight_interaction,horsepower_binned,body-style_hardtop,body-style_hatchback,body-style_sedan,body-style_wagon
3,2,164,audi,gas,std,four,fwd,front,99.8,176.6,...,5500,24,30,13950,254733,Medium,False,False,True,False
4,2,164,audi,gas,std,four,4wd,front,99.4,176.6,...,5500,18,22,17450,384064,Medium,False,False,True,False
6,1,158,audi,gas,std,four,fwd,front,105.8,192.7,...,5500,19,25,17710,386784,Medium,False,False,True,False
8,1,158,audi,gas,turbo,four,fwd,front,105.8,192.7,...,5500,17,20,23875,404266,Medium,False,False,True,False
10,2,192,bmw,gas,std,two,rwd,front,101.2,176.8,...,5800,23,29,16430,258660,Medium,False,False,True,False


In [20]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder

In [21]:
# Label encode the 'body-style' column
le = LabelEncoder()
df['body-style_encoded'] = le.fit_transform(df['body-style'])

In [22]:
# Display the label-encoded feature
print("Label Encoded Feature (body-style):")
print(df[['body-style', 'body-style_encoded']].head())

Label Encoded Feature (body-style):
   body-style  body-style_encoded
3       sedan                   3
4       sedan                   3
6       sedan                   3
8       sedan                   3
10      sedan                   3


In [23]:
# Encoding categorical columns
label_encoders = {}

for column in df.select_dtypes(include=['object']).columns:
 le = LabelEncoder()
 df[column] = le.fit_transform(df[column])
 label_encoders[column] = le
    
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,engine_curb_weight_interaction,horsepower_binned,body-style_encoded
3,2,27,0,1,0,0,3,1,0,99.8,...,3.4,10.0,102,5500,24,30,13950,254733,Medium,3
4,2,27,0,1,0,0,3,0,0,99.4,...,3.4,8.0,115,5500,18,22,17450,384064,Medium,3
6,1,25,0,1,0,0,3,1,0,105.8,...,3.4,8.5,110,5500,19,25,17710,386784,Medium,3
8,1,25,0,1,1,0,3,1,0,105.8,...,3.4,8.3,140,5500,17,20,23875,404266,Medium,3
10,2,31,1,1,0,1,3,2,0,101.2,...,2.8,8.8,101,5800,23,29,16430,258660,Medium,3


### Dimensionality Reduction (Optional)

In [25]:
from sklearn.decomposition import PCA

In [27]:
# Select numerical features for PCA
numerical_features = df[['curb-weight', 'engine-size', 'horsepower', 'city-mpg', 'highway-mpg']]

In [28]:
# Apply PCA
pca = PCA(n_components=2) # Reduce to 2 dimensions
pca_features = pca.fit_transform(numerical_features)

In [29]:
# Display the reduced features
print("Reduced Features using PCA:")
print(pca_features[:5])

Reduced Features using PCA:
[[-1.23989885e+02  1.04557656e+01]
 [ 3.63886516e+02  3.73556650e-01]
 [ 3.83534750e+02 -5.90427734e+00]
 [ 6.26123462e+02  2.34310236e+00]
 [-6.62468467e+01  5.24056145e+00]]


In [30]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,engine_curb_weight_interaction,horsepower_binned,body-style_encoded
3,2,27,0,1,0,0,3,1,0,99.8,...,3.4,10.0,102,5500,24,30,13950,254733,Medium,3
4,2,27,0,1,0,0,3,0,0,99.4,...,3.4,8.0,115,5500,18,22,17450,384064,Medium,3
6,1,25,0,1,0,0,3,1,0,105.8,...,3.4,8.5,110,5500,19,25,17710,386784,Medium,3
8,1,25,0,1,1,0,3,1,0,105.8,...,3.4,8.3,140,5500,17,20,23875,404266,Medium,3
10,2,31,1,1,0,1,3,2,0,101.2,...,2.8,8.8,101,5800,23,29,16430,258660,Medium,3


In [31]:
# Convert PCA features to a DataFrame with appropriate column names
pca_df = pd.DataFrame(pca_features, columns=['PCA1', 'PCA2'])

In [32]:
# Add the PCA features back to the original dataframe
df = pd.concat([df, pca_df], axis=1)
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,horsepower,peak-rpm,city-mpg,highway-mpg,price,engine_curb_weight_interaction,horsepower_binned,body-style_encoded,PCA1,PCA2
3,2.0,27.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,99.8,...,102.0,5500.0,24.0,30.0,13950.0,254733.0,Medium,3.0,626.123462,2.343102
4,2.0,27.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,99.4,...,115.0,5500.0,18.0,22.0,17450.0,384064.0,Medium,3.0,-66.246847,5.240561
6,1.0,25.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,105.8,...,110.0,5500.0,19.0,25.0,17710.0,386784.0,Medium,3.0,252.001751,24.682154
8,1.0,25.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,105.8,...,140.0,5500.0,17.0,20.0,23875.0,404266.0,Medium,3.0,-976.362946,-2.551792
10,2.0,31.0,1.0,1.0,0.0,1.0,3.0,2.0,0.0,101.2,...,101.0,5800.0,23.0,29.0,16430.0,258660.0,Medium,3.0,-553.677250,1.428452


In [35]:
# Drop the original numerical columns to reduce complexity
df.drop(columns=['curb-weight', 'engine-size', 'horsepower', 'city-mpg', 'highway-mpg'], inplace=True)

KeyError: "['curb-weight', 'engine-size', 'horsepower', 'city-mpg', 'highway-mpg'] not found in axis"

In [36]:
# Display the updated dataframe
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,bore,stroke,compression-ratio,peak-rpm,price,engine_curb_weight_interaction,horsepower_binned,body-style_encoded,PCA1,PCA2
3,2.0,27.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,99.8,...,3.19,3.4,10.0,5500.0,13950.0,254733.0,Medium,3.0,626.123462,2.343102
4,2.0,27.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,99.4,...,3.19,3.4,8.0,5500.0,17450.0,384064.0,Medium,3.0,-66.246847,5.240561
6,1.0,25.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,105.8,...,3.19,3.4,8.5,5500.0,17710.0,386784.0,Medium,3.0,252.001751,24.682154
8,1.0,25.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,105.8,...,3.13,3.4,8.3,5500.0,23875.0,404266.0,Medium,3.0,-976.362946,-2.551792
10,2.0,31.0,1.0,1.0,0.0,1.0,3.0,2.0,0.0,101.2,...,3.50,2.8,8.8,5800.0,16430.0,258660.0,Medium,3.0,-553.677250,1.428452


In [39]:
# Select the columns to apply PCA again (excluding non-numericalcolumns)
columns_for_pca = df.select_dtypes(include=['float64','int64']).columns

columns_for_pca

Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'engine-type',
       'num-of-cylinders', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'peak-rpm', 'price',
       'engine_curb_weight_interaction', 'body-style_encoded', 'PCA1', 'PCA2'],
      dtype='object')

In [40]:
# Impute missing values with the median of each column
df[columns_for_pca] = df[columns_for_pca].fillna(df[columns_for_pca].median())

In [41]:
# Apply PCA again on the dataset
pca = PCA(n_components=2) # Reduce to 2 dimensions again
pca_features = pca.fit_transform(df[columns_for_pca])

In [42]:
# Convert PCA features to a DataFrame with appropriate column names
pca_df = pd.DataFrame(pca_features, columns=['PCA1_reduced', 'PCA2_reduced'])

In [43]:
# Add the new PCA features back to the original dataframe
df = pd.concat([df, pca_df], axis=1)

In [44]:
# Drop the original columns that were used for the second PCA
df.drop(columns=columns_for_pca, inplace=True)

In [45]:
# Display the updated dataframe with reduced columns
df.head()

,horsepower_binned,PCA1_reduced,PCA2_reduced
3,Medium,107635.576769,8998.080721
4,Medium,-38145.232619,6859.322645
6,Medium,147678.145416,4589.583336
8,Medium,-206336.043054,1661.782209
10,Medium,-125296.280202,169.711065
